In [1]:
from matrix import *
import numpy as np
import math
import random
from typing import List, Tuple
random.seed(57)

def generate_matrix(rows, cols, min_val, max_val):
    return save_matrix([[random.randint(min_val, max_val) for i in range(cols)] for j in range(rows)])

# Expert 1
**Задача**:

Необходимо доказать, что оптимальные направления PCA совпадают с собственными векторами матрицы ковариаций, соответствующими максимальным собственным значениям.

Поиск оптимальных направлений означает, что необходимо найти направления, максимизирующие дисперсию. Формулы для неё и для ковариации задаются следующим образом:

Ковариация: $Cov (X, Y) = (X, Y) = \frac{1}{n-1} X^T Y$

Дисперсия: $Var(X) = ||X||^2 = (X, X) = \frac{1}{n-1} X^T X$

----
**Доказательство**:

1. Матрица ковариаций имеет следующий вид: $$\Sigma = \frac{1}{n-1}X^T X,$$ где $X$ - центрированная матрица данных размерности $n \times m$, а $n$ - количество объектов.

    Пусть $W$ - искомые $k$ векторов, $W = (w_1, ..., w_k)$, причём $||w_i|| = 1$ (векторы нормированны, то есть имеют единичные длины). Тогда проекция данных на $W$ – это $XW$, и она должна иметь максимальную дисперсию.

    $Var(XW) = ||XW||^2 = \frac{1}{n-1} (XW)^T XW = \frac{1}{n-1} W^TX^T XW = W^T \Sigma W$.

    Получаем, что необходимо максимизировать $Var(XW) = W^T \Sigma W$.

    Пусть $w_1$ - первая главная компонента. Сначала найдём $max(w_1^T \Sigma w_1$), а затем оставшиеся $w_2, ..., w_k$, которые должны быть ортогональны предыдущим (то есть $w_i^T w_j = 0$ при $i \neq j$).

2. Воспользуемся сингулярным разложением матрицы $X$: $X = USV^T$, где $U$ - матрица левых сингулярных векторов (является ортогональной), $V$ - матрица правых сингулярных векторов (также является ортогональной), $S$ - диагональная матрица сингулярных чисел. $$\Sigma = \frac{1}{n-1} X^TX = \frac{1}{n-1} (USV^T)^T USV = \frac{1}{n-1} VS^T U^T USV = \frac{1}{n-1} V S^T E SV^T = \frac{1}{n-1} V S^T SV^T$$

    Пусть $\Lambda = \frac{1}{n-1} S^T S$, тогда $\Sigma = V \Lambda V^T$.

    Получаем спектральное разложение матрицы $\Sigma$. $\Lambda$ диагональна, на диагонали находятся собственные значения матрицы, а $V$ состоит из собственных векторов.

3. Величина, которую нужно максимизировать, теперь выражается как $max(w_1^T (V \Lambda V^T) w_1), ||w_1|| = 1, w_1$ - искомый.

    Замена: $y = V^T w_1$, откуда $w_1 = V \cdot y$. Так как $V$  ортогональна, получаем: $||w_1|| = ||Vy|| = ||y|| = 1$.

    Теперь ищем: $max \big[ (Vy)^T (V \Lambda V^T) V y \big]$ при $||y|| = 1$. преобразуем выражение:

    $$(Vy)^T (V \Lambda V^T) Vy = y^T \cdot V^T V \cdot \Lambda \cdot V^T V \cdot y = y^T \cdot E \cdot E \cdot y = y^T  \Lambda y = \sum^n_{i=1} \lambda_i y_i^2$$

    При этом из условия $||y|| = 1$ следует, что $$\sum^n_{i=1} y^2_i = 1$$

4. Искомый максимум: $max(\lambda_1 y_1^2 + \lambda_2 y_2^2 + ... + \lambda_n y_n^2)$. Так как предполагается, что $\lambda_1, ..., \lambda_n$ отсортированы по невозрастанию, то максимум будет достигаться при $y_1 = 1$, а остальные $y_i = 0$.

    $y = (1, 0, ..., 0)^T$, тогда $w_1 = Vy = V \cdot (1, 0, ..., 0)^T = V^{(1)}$ - первый столбец $V$.

    $V^{(1)}$ - в точности собственный вектор, соответствующий максимальному собственному значению, то есть для первой главной компоненты $w_1$ всё доказано.

5. Теперь приведём рассуждения о выборе следующих компонент ($w_2, ..., w_k$). При поиске $w_2$ следует учитывать, что он и $w_1$ должны быть ортогональны:

    Пусть $w_1 = Vy_1$, где $y_1 = (1, 0, ..., 0)^T$, а $w_2 = Vy_2$.

    $$(w_1, w_2) = (Vy_1, Vy_2) = 0 \Rightarrow (Vy_1)^T Vy_2 = y_1^T V^T V y_2 = y_1^T y_2 = 0 $$

    Так как $(1, 0, ..., 0)^T y_2$ = 0, то первая компонента $y_2$ нулевая, остальные любые. Тогда из $max(\lambda_1 y_1^2 + \lambda_2 y_2^2 + ... + \lambda_n y_n^2)$ можем получить $max( \lambda_2 y_2^2 + ... + \lambda_n y_n^2)$, и аналогично поиску для $w_1$ находим $y_2 = (0, 1, 0, ..., 0)^T$, откуда $w_2 = V^{(2)}$ (нашли второй столбец и второй собственный вектор).

Продолжив алгоритм, получим искомые $w_1, w_2, ..., w_k$, равные собственным векторам матрицы $\Sigma$, соответствующие в точности максимальным собственным значениям. Таким образом, утверждение доказано.

# Easy 1: Метод Гаусса

In [2]:
def gauss_solver(A: Matrix, b: Matrix, tol: float = 1e-5) -> List[Matrix]:
    if A.rows != b.rows or b.columns != 1:
        raise ValueError("Размеры A и b не согласованы")

    n, m = A.rows, A.columns
    # преобразуем в плотный формат
    classic_A = get_classic_format(A)
    classic_b = get_classic_format(b)

    Ab = [row.copy() + [classic_b[i][0]] for i, row in enumerate(classic_A)]

    rank = 0
    # Прямой ход Гаусса
    for col in range(m):
        max_row = rank
        for row in range(rank + 1, n):
            if abs(Ab[row][col]) > abs(Ab[max_row][col]):
                max_row = row

        # Если максимальный элемент равен нулю, скипаем столбец
        if abs(Ab[max_row][col]) < tol:
            continue

        # Меняем строки местами
        Ab[rank], Ab[max_row] = Ab[max_row], Ab[rank]

        # Нормализация строки
        pivot = Ab[rank][col]
        for j in range(col, m + 1):
            Ab[rank][j] /= pivot

        # Исключаем переменную из других строк
        for i in range(n):
            if i != rank and abs(Ab[i][col]) > tol:
                factor = Ab[i][col]
                for j in range(col, m + 1):
                    Ab[i][j] -= factor * Ab[rank][j]

        rank += 1
        if rank == n:
            break

    # Проверяем совместность
    for i in range(rank, n):
        if abs(Ab[i][m]) > tol:
            raise ValueError("Система несовместна")

    # Обратный ход Гаусса
    solutions = []
    free_vars = []
    lead_vars = [-1] * m

    # Определяем свободные переменные
    for i in range(rank):
        for j in range(m):
            if abs(Ab[i][j]) > tol:
                lead_vars[j] = i
                break

    for j in range(m):
        if lead_vars[j] == -1:
            free_vars.append(j)

    # Если свободных нет - единственное решение
    if not free_vars:
        solution = [[0.0] for _ in range(m)]
        for i in range(rank):
            for j in range(m):
                if abs(Ab[i][j]) > tol:
                    solution[j][0] = Ab[i][m]
                    break

        sol = [(i, 0, solution[i][0]) for i in range(m) if abs(solution[i][0]) > tol]
        return [Matrix(m, 1, sol)]

    # Если свободные есть - строим базис
    for free in free_vars:
        vec = [[0.0] for _ in range(m)]
        vec[free][0] = 1.0

        for i in range(rank):
            for j in range(m):
                if abs(Ab[i][j]) > tol:
                    sum_ax = 0.0
                    for k in range(j + 1, m):
                        sum_ax += Ab[i][k] * vec[k][0]
                    vec[j][0] = Ab[i][m] - sum_ax
                    break

        basis = [(i, 0, vec[i][0]) for i in range(m) if abs(vec[i][0]) > tol]
        solutions.append(Matrix(m, 1, basis))

    return solutions

In [3]:
# Пример системы уравнений:
# 1x + 2y = 5
# 3x + 4y = 11

A = save_matrix([[1, 2], [3, 4]])
b = save_matrix([[5], [11]])

solutions = gauss_solver(A, b)
if len(solutions) == 1:
    print("Единственное решение:")
    print(solutions[0])
else:
    print(f"Бесконечно много решений. Базис ФСР (размерность {len(solutions)}):")
    for i, sol in enumerate(solutions):
        print(f"Решение {i+1}:")
        print(sol)

Единственное решение:
1.0
2.0


# Easy 2: Центрирование данных
$$ X_{centered} = X - X_{mean} $$

In [4]:
def get_means(X: Matrix) -> List[float]:
    rows = X.rows
    cols = X.columns
    means = [0.0] * cols

    for col in range(1, cols + 1):
        for row in range(1, rows + 1):
            means[col-1] += X.get(row, col)
        means[col-1] /= rows

    return means

def center_data(X: Matrix) -> Matrix:
    means = get_means(X)
    centered_data = []

    for row in range(1, X.rows + 1):
        for col in range(1, X.columns + 1):
            val = X.get(row, col)
            centered_val = val - means[col-1]
            centered_data.append((row-1, col-1, centered_val))

    return Matrix(X.rows, X.columns, centered_data)

Сравнение с NumPy:

In [5]:
X = generate_matrix(3, 3, -3, 3)

print("Исходная матрица:")
print(X)

X_centered = center_data(X)
print("\nНаше центрирование:")
print(X_centered)

X_np = np.array(get_classic_format(X))
centered_np = X_np - X_np.mean(axis=0)

print("\nNumPy центрирование:")
print(centered_np)


Исходная матрица:
-3.0 -1.0 1.0
1.0 -3.0 -2.0
1.0 -1.0 0

Наше центрирование:
-2.6666666666666665 0.6666666666666667 1.3333333333333333
1.3333333333333333 -1.3333333333333333 -1.6666666666666667
1.3333333333333333 0.6666666666666667 0.3333333333333333

NumPy центрирование:
[[-2.66666667  0.66666667  1.33333333]
 [ 1.33333333 -1.33333333 -1.66666667]
 [ 1.33333333  0.66666667  0.33333333]]


# Easy 3: матрица ковариаций
$$ C = \frac{1}{n-1}X^TX $$

In [6]:
def covariance_matrix(X_centered: Matrix) -> Matrix:
    n = X_centered.rows
    p = X_centered.columns
    X = get_classic_format(X_centered)

    cov_data = []
    for i in range(p):
        for j in range(p):
            cov = sum(X[row][i] * X[row][j] for row in range(n)) / (n - 1)
            cov_data.append((i, j, cov))

    return Matrix(p, p, cov_data)

Сравнение с NumPy:

In [11]:
X = generate_matrix(5, 5, -10, 10)

X_centered = center_data(X)
cov = covariance_matrix(X_centered)

X_np = np.array(get_classic_format(X))
cov_numpy = np.cov(X_np, rowvar=False, bias=False)

print("Наш результат:")
print(cov)
print("\nNumPy результат:")
print(cov_numpy)


Наш результат:
60.7 20.150000000000006 -1.8000000000000025 56.0 -14.099999999999996
20.150000000000006 40.300000000000004 -18.6 15.500000000000004 14.3
-1.8000000000000025 -18.6 17.2 -2.7500000000000018 2.6499999999999995
56.0 15.500000000000004 -2.7500000000000018 56.5 -8.999999999999998
-14.099999999999996 14.3 2.6499999999999995 -8.999999999999998 64.3

NumPy результат:
[[ 60.7   20.15  -1.8   56.   -14.1 ]
 [ 20.15  40.3  -18.6   15.5   14.3 ]
 [ -1.8  -18.6   17.2   -2.75   2.65]
 [ 56.    15.5   -2.75  56.5   -9.  ]
 [-14.1   14.3    2.65  -9.    64.3 ]]


# Normal 1: Собственные значения

In [12]:
# поиск корня бисекцией
def root_search(f, a, b, tol=1e-6):
    fa, fb = f(a), f(b)
    if fa * fb > 0:
        return None

    while (b-a) > tol:
        c = (a + b) / 2
        fc = f(c)
        if abs(fc) < tol:
            return c
        if fa * fc < 0:
            b, fb = c, fc
        else:
            a, fa = c, fc
    return (a + b) / 2

# поиск экстремума бисекцией
def extremum_search(f, a0, b0, epsilon, delta=1e-10):
    a = a0
    b = b0
    ans = (a + b) / 2

    while abs(b - a) > epsilon:
        yk = (a + b - delta) / 2
        zk = (a + b + delta) / 2

        if f(yk) <= f(zk):
            b = zk
        else:
            a = yk

        ans = (a + b) / 2
    return ans

In [13]:
# функция определителя
def determinant(mat):
    n = len(mat)
    if n == 1:
        return mat[0][0]
    if n == 2:
        return mat[0][0]*mat[1][1] - mat[0][1]*mat[1][0]

    det = 0
    for col in range(n):
        minor = [row[:col] + row[col+1:] for row in mat[1:]]
        det += (-1)**col * mat[0][col] * determinant(minor)

    return det

In [121]:
# основная функция. поиск собственных значений
def find_eigenvalues(C_matrix: 'Matrix', tol: float = 1e-6) -> List[float]:
    n = C_matrix.rows
    C = get_classic_format(C_matrix)

# обычный характеристический полином
    def characteristic_poly(lam):
        mat = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
        return determinant(mat)
        # чтоб оперативней считалось, потом закомментить
        #return np.linalg.det(mat)

    # модуль полинома для корней чётной кратности
    def poly_abs(lam):
        mat = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
        return abs(determinant(mat))
        # чтоб оперативней считалось, потом закомментить
        #return abs(np.linalg.det(mat))

    # Вычисляем интервал поиска (см теорему Гершгорина)
    Gershgorin_intervals = []
    for i in range(n):
        radius = sum(abs(C[i][j]) for j in range(n) if j != i)
        center = C[i][i]
        Gershgorin_intervals.append((center - radius, center + radius))

    lower = min(interval[0] for interval in Gershgorin_intervals)
    upper = max(interval[1] for interval in Gershgorin_intervals)

    # Расширяем интервал, ну чтобы понадёжнее
    lower -= 3
    upper += 3
    count = 2*math.ceil(upper - lower)

    # непосредственно поиск корней
    def find_roots(f, fabs, a, b, tol, count):
        roots = []
        # дробим [a,b] на мелкие подотрезки
        step = (b - a) / count

        x_prev = a
        f_prev = f(x_prev)

        # пробегаем по каждому маленькому отрезку
        for i in range(1, count+1):
            x = a + i * step
            fx = f(x)

            # если попали в корень
            if abs(fx) < tol:
                roots.append(x)
            # если функция меняет знак, ищем корень
            # если не меняет, смотрим экстремум для функции-модуля.
            if f_prev * fx < 0:
                roots.append(root_search(f, x_prev, x, tol))
            else:
                exst = extremum_search(fabs, x_prev, x, tol)
                # если при поиске экстремума «скатились» близко к нулю, то всё ок
                if abs(f(exst)) < 0.0001:
                  roots.append(exst)

            x_prev = x
            f_prev = fx

        # убираем дубликаты (близкие корни)
        unique_roots = [roots[0]]
        for i in range(1,len(sorted(roots))):
            if not unique_roots or (roots[i] - roots[i-1] > 2*step):
                unique_roots.append(roots[i])

        return unique_roots

    eigenvalues = find_roots(characteristic_poly, poly_abs, lower, upper, tol, count)
    return sorted(eigenvalues)[::-1]

In [122]:
C = generate_matrix(5, 5, -10, 10)
cov = covariance_matrix(C)
print("Исходная матрица:")
print(cov)

eigenvalues = find_eigenvalues(cov)
print("\nНайденные собственные значения:")
for i, val in enumerate(eigenvalues, 1):
    print(f"lambda{i} = {val:.6f}")

np_matrix = get_classic_format(cov)
np_eigenvalues = np.linalg.eigvals(np_matrix)
print("\nПроверка с numpy:")
for i, val in enumerate(sorted(np_eigenvalues, reverse=True), 1):
    print(f"lambda{i} = {val:.6f}")

Исходная матрица:
54.0 26.75 -0.25 13.25 45.0
26.75 39.0 -2.5 8.75 4.5
-0.25 -2.5 42.0 -0.25 15.0
13.25 8.75 -0.25 49.25 -29.5
45.0 4.5 15.0 -29.5 88.75

Найденные собственные значения:
lambda1 = 128.176623
lambda2 = 76.464960
lambda3 = 42.431568
lambda4 = 25.604125
lambda5 = 0.322725

Проверка с numpy:
lambda1 = 128.176623
lambda2 = 76.464959
lambda3 = 42.431568
lambda4 = 25.604126
lambda5 = 0.322725


# Normal 2: Собственные векторы

In [94]:
def normalize_vector(vect: Matrix) -> Matrix:
    vector = get_classic_format(vect)
    norm = math.sqrt(sum(x[0] ** 2 for x in vector))
    return save_matrix([[x[0] / norm] for x in vector])

def find_eigenvectors(C_matrix: Matrix, eigenvalues: List[float], tol = 1e-2) -> List[Matrix]:
    n = C_matrix.rows
    vectors_list = []

    C = get_classic_format(C_matrix)
    for lam in eigenvalues:
        C_lam = save_matrix([[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)])
        vectors_lam = gauss_solver(C_lam, save_matrix([[0]]*n), tol)
        vectors_list += vectors_lam

    normalize_list = []
    for vectors in vectors_list:
        normalize_list.append(normalize_vector(vectors))
    return normalize_list

Сравнение с NumPy:

In [123]:
# C = save_matrix([[-3, 18, 9, 50, 80],
#                [7, -86, -25, -15, 94],
#                [-65, 6, -66, 18, 67],
#                [22, -1, -39, -41, 87],
#                [-85, -3, 26, 2, 22]])
C = generate_matrix(5, 5, -10, 10)

X_centered = center_data(C)
covar = covariance_matrix(X_centered)
vals = find_eigenvalues(covar)
vects = find_eigenvectors(covar, vals)

np_covar = np.cov(get_classic_format(X_centered), rowvar=False)
np_vals, np_vects = np.linalg.eig(np_covar)

sorted_indices = np.argsort(np_vals)[::-1]
np_vals_sorted = np_vals[sorted_indices]
np_vects_sorted = np_vects[:, sorted_indices]

# Сравнение
print("Собственные значения:")
print("Наши:", vals)
print("NumPy:", np_vals_sorted)

print("\nСобственные вектора:")
print("Наши:\n", vects)
print("NumPy\n:", np_vects_sorted)

Собственные значения:
Наши: [99.45672744054062, 15.348640906871884, 3.26054374046814, 4.6130937475217674e-07]
NumPy: [ 9.94567271e+01  1.53486408e+01  3.26054331e+00  9.34088843e-01
 -8.35734648e-16]

Собственные вектора:
Наши:
 [0.49203873963286726
0.7474380806495947
0.2708638178899875
-0.16729512427812532
0.31285672095413825, 0.12451349156425087
-0.4345654172168424
-0.16328237929659306
-0.522060157955868
0.7045859384517752, -0.7595538058544158
0.21497549542310676
0.4538950671573336
-0.4073269756008807
0.07019654790473348, -0.40377571513915167
0.33490815769857873
-0.49190890514466334
0.4697738257048034
0.5119959760514355]
NumPy
: [[-0.49203874 -0.1245135  -0.75955378  0.04937153  0.40377565]
 [-0.74743808  0.43456542  0.21497547 -0.30677545 -0.33490833]
 [-0.27086382  0.16328238  0.45389511  0.67229161  0.49190942]
 [ 0.16729513  0.52206016 -0.407327    0.55934167 -0.4697735 ]
 [-0.31285672 -0.70458593  0.07019651  0.37230655 -0.51199572]]


# Normal 3: Доля объяснённой дисперсии

In [109]:
def explained_variance_ratio(eigenvalues: List[float], k: int) -> float:
    return sum(eigenvalues[:k]) / sum(eigenvalues)

Сравнение с NumPy:

In [111]:
# C = save_matrix([[-3, 18, 9, 50, 80],
#                [7, -86, -25, -15, 94],
#                [-65, 6, -66, 18, 67],
#                [22, -1, -39, -41, 87],
#                [-85, -3, 26, 2, 22]])

C = generate_matrix(5, 5, -10, 10)
X_centered = center_data(C)
covar = covariance_matrix(X_centered)
vals = find_eigenvalues(covar)


np_covar = np.cov(get_classic_format(X_centered), rowvar=False)
np_vals = np.linalg.eigvals(np_covar)
np_vals_sorted = sorted(np_vals, reverse=True)

for k in [1, 3, 5]:
    our_ratio = explained_variance_ratio(vals, k)

    np_ratio = sum(np_vals_sorted[:k]) / sum(np_vals_sorted)

    print(f"k={k}:")
    print(f"Наш результат: {our_ratio:.6f}")
    print(f"NumPy результат: {np_ratio:.6f}\n")

k=1:
Наш результат: 0.626057
NumPy результат: 0.626057

k=3:
Наш результат: 0.984351
NumPy результат: 0.984351

k=5:
Наш результат: 1.000000
NumPy результат: 1.000000



# Hard 1: алгоритм PCA

In [118]:
def project_data(X_matrix: Matrix, eigenvectors: List[Matrix], k: int) -> Tuple[Matrix, Matrix]:
    n = X_matrix.rows
    X = get_classic_format(X_matrix)
    vector_matrix = []

    for i in range(n):
        row = []
        for v in eigenvectors[:k]:
            row.append(v.get(i + 1, 1))
        vector_matrix.append(row)

    vector_matrix = save_matrix(vector_matrix)
    projected_X = multiply(X_matrix, vector_matrix)

    return projected_X, vector_matrix

def pca(X: Matrix, k: int) -> Tuple[Matrix, float]:
    # центрирование данных и матрица ковариаций
    centered_X = center_data(X)
    covar_X = covariance_matrix(centered_X)

    # поиск собственных значений и векторов
    eigenvals = find_eigenvalues(covar_X)
    eigenvects = find_eigenvectors(covar_X, eigenvals)

    # проекция данных на главные компоненты
    projected_X = project_data(centered_X, eigenvects, k)
    dispersion_res = explained_variance_ratio(eigenvals, k)

    return projected_X[0], dispersion_res

In [125]:
from sklearn.decomposition import PCA

# C = save_matrix([[-3, 18, 92, 50, 80], [67, -86, -25, -15, 94], [-65, 60, -66, 18, 67], [22, -1, -39, -41, 87], [-85, -3, 26, 2, 22]])
C = generate_matrix(5, 5, -25, 25)
my_res = pca(C, 3)[0]
print(f"Наш результат: {my_res}")

pca_import = PCA(n_components=3)
data_pca = pca_import.fit_transform(get_classic_format(C))
print(f"\nСравнение с sklearn: {data_pca}")


Наш результат: 24.941959804225714 -20.900365856429328 -11.49047852865236
-31.30961918892793 11.650999019481228 -11.589791770530097
20.677745818853978 29.535223340337048 3.512005655591781
0.3137426009393711 -1.42555373644688 9.189290374314968
-14.62382903509114 -18.860302766942066 10.378974269275709

Сравнение с sklearn: [[ 24.9419598   20.90036585 -11.49047854]
 [-31.30961919 -11.65099903 -11.58979176]
 [ 20.67774582 -29.53522334   3.51200566]
 [  0.3137426    1.42555374   9.18929039]
 [-14.62382904  18.86030278  10.37897425]]
